# 1

思路参考之前的10题。

递归判断。

*可消耗，可不消耗。\
?必须消耗。

注意：这是'通配符'，而非正则表达式。\
'*'和'?'也作为字符匹配，而不是表示前面字符的数量。

超时！

In [74]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        if not p:   # p为空
            return not s
        if len(p) >= 2 and p[0] == '*' and p[1] == '*':    # 跳过p中连着的*
            return self.isMatch(s, p[1:])
        if not s:   # s为空
            return not p or p[0] == '*' and self.isMatch(s, p[1:])   # s没了，p还有，继续匹配p是否为*

        first_match = bool(s) and p[0] in ('?', s[0])

        if p[0] == '*':
            return (self.isMatch(s, p[1:]) or       # 匹配0个
                    self.isMatch(s[1:], p[1:]) or   # 匹配1个
                    self.isMatch(s[1:], p))         # 匹配1个以上
        else:
            return first_match and self.isMatch(s[1:], p[1:])


if __name__ == '__main__':
    s_p = [
        ['aa', 'a'],
        ['aa', '*'],
        ['cb', '?a'],
        ['adceb', 'a*b'],
        ['acdcb', 'a*c?b'],
        ['', '******'],
        ['babaaababaabababbbbbbaabaabbabababbaababbaaabbbaaab', '***bba**a*bbba**aab**b'],
        ['aa', '*'],
        ['aa', '?*'],
        ['abaabaaaabbabbaaabaabababbaabaabbabaaaaabababbababaabbabaabbbbaabbbbbbbabaaabbaaaaabbaabbbaaaaabbbabb', 'ab*aaba**abbaaaa**b*b****aa***a*b**ba*a**ba*baaa*b*ab*'],
        ['bbbbbbbabbaabbabbbbaaabbabbabaaabbababbbabbbabaaabaab', 'b*b*ab**ba*b**b***bba'], # 超时
    ]

    for s, p in s_p:
        print(Solution().isMatch(s, p))

False
True
False
True
False
True
False
True
True
False
False


# 题解1



动态规划

字符串匹配，只往前走，只能往右下角走。

![1](https://pic.leetcode-cn.com/6d81e12e8aa6f3ff5bf08dcd602fed756f4cfe9fceeba7c5b7c01b3e7c87f1f5-%E6%B0%B4%E5%8D%B0_%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7%202020-07-05%20%E4%B8%8B%E5%8D%882.03.39.png)
![2](https://pic.leetcode-cn.com/a319e64f7824ab0590ef1dbaa016d6e47f22c631424b124cb1ecea842ba002c8-gaitubao_%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7%202020-07-05%20%E4%B8%8B%E5%8D%882.09.44.png)
![3](https://pic.leetcode-cn.com/c97b033d8e3f45686f87a3264404177411ac8095201c07c3e1af0b25953067b0-gaitubao_%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7%202020-07-05%20%E4%B8%8B%E5%8D%882.20.13.png)
![4](https://pic.leetcode-cn.com/abee7c42685e277f93c1693447d403044e30a33d1a90f763a96d3373ed66ce2f-gaitubao_%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7%202020-07-05%20%E4%B8%8B%E5%8D%882.44.25.png)
![5](https://pic.leetcode-cn.com/401a04aafc325886cb448e3d9c0c78aa223f2387e3512ff17f9966daef15d705-gaitubao_%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7%202020-07-05%20%E4%B8%8B%E5%8D%882.25.03.png)
![6](https://pic.leetcode-cn.com/79bb8ed8011d87781e87346361f44547d0b162594639ef81b38d2aaeda05f162-gaitubao_%E5%B1%8F%E5%B9%95%E5%BF%AB%E7%85%A7%202020-07-05%20%E4%B8%8B%E5%8D%882.26.42.png)

In [122]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        s, p = ' ' + s, ' ' + p # 哨
        m, n = len(s), len(p)
        dp = [[False for _ in range(m)] for _ in range(n)]

        dp[0][0] = True

        for i in range(1, n):   # 题解抄来，单独处理*开头的情况（后面就不再管了）
            if p[i] == '*':
                dp[i][0] = True
            else:
                break

        for i in range(1, n):
            for j in range(1, m):
                # dp[i][j] = (
                #     (p[i] in (s[j], '?') and (dp[i-1][j-1] or (p[i-1] == '*' and dp[i-1][j])))
                #     or (p[i] == '*' and (dp[i-1][j] or dp[i][j-1] or dp[i-1][j-1]))
                # )
                dp[i][j] = (
                    (p[i] in (s[j], '?') and (dp[i-1][j-1]))                        # 当前字符相等或为?，并且左上角dp为T
                    or (p[i] == '*' and (dp[i-1][j] or dp[i][j-1] or dp[i-1][j-1])) # 当前p为*，并且上面/左边/左上角为T
                )
        
        # print('\t'.join(list(s)))     # debug
        # for i in range(n):
        #     print(p[i], end='')
        #     for j in range(m):
        #         print(f'{"T" if dp[i][j] else " "}', end='\t')
        #     print()
        return dp[-1][-1]


if __name__ == '__main__':
    s_p = [
        # ['aa', 'a'],
        # ['aa', '*'],
        # ['cb', '?a'],
        # ['aa', '?a'],
        # ['adceb', 'a*b'],
        # ['acdcb', 'a*c?b'],
        # ['', '******'],
        # ['babaaababaabababbbbbbaabaabbabababbaababbaaabbbaaab', '***bba**a*bbba**aab**b'],
        # ['aa', '*'],
        # ['aa', '?*'],
        # ['abaabaaaabbabbaaabaabababbaabaabbabaaaaabababbababaabbabaabbbbaabbbbbbbabaaabbaaaaabbaabbbaaaaabbbabb', 'ab*aaba**abbaaaa**b*b****aa***a*b**ba*a**ba*baaa*b*ab*'],
        # ['bbbbbbbabbaabbabbbbaaabbabbabaaabbababbbabbbabaaabaab', 'b*b*ab**ba*b**b***bba'],
        # ['adceb', '*a*b'],
        # ['aab', 'c*a*b'],
        # ['ab', '?*'],
        ['mississippi', 'm??*ss*?i*pi'],    # 不单独处理开头，而是每次都处理上面是*的问题，导致dp[5][4]为T，依次错下去
        ['adceb', '*a*b'],
    ]

    for s, p in s_p:
        print(Solution().isMatch(s, p))

 	m	i	s	s	i	s	s	i	p	p	i
 T	 	 	 	 	 	 	 	 	 	 	 	
m 	T	 	 	 	 	 	 	 	 	 	 	
? 	 	T	 	 	 	 	 	 	 	 	 	
? 	 	 	T	 	 	 	 	 	 	 	 	
* 	 	 	T	T	T	T	T	T	T	T	T	
s 	 	 	 	T	 	T	T	 	 	 	 	
s 	 	 	 	 	 	 	T	 	 	 	 	
* 	 	 	 	 	 	 	T	T	T	T	T	
? 	 	 	 	 	 	 	 	T	T	T	T	
i 	 	 	 	 	 	 	 	 	 	 	T	
* 	 	 	 	 	 	 	 	 	 	 	T	
p 	 	 	 	 	 	 	 	 	 	 	 	
i 	 	 	 	 	 	 	 	 	 	 	 	
False
 	a	d	c	e	b
 T	 	 	 	 	 	
*T	T	T	T	T	T	
a 	T	 	 	 	 	
* 	T	T	T	T	T	
b 	 	 	 	 	T	
True


执行用时 :760 ms, 在所有 Python3 提交中击败了30.47 %的用户

内存消耗 :23.5 MB, 在所有 Python3 提交中击败了19.97 %的用户

time: O(mn)

space: O(mn)

# 题解2

贪心算法

分三种情况：\
模式p的开头和结尾是∗； （如：p=∗abcd∗efgh∗i∗）\
模式p的开头字符不是∗；（如：p=abcd∗efgh∗i∗）\
模式p的结尾字符不是∗。（如：p=∗abcd∗efgh∗i）

与动态规划相比，空间复杂度降为O(1)。

p=∗abcd∗ 为例，pp 可以匹配所有包含子串 abcd 的字符串，也就是说，我们只需要暴力地枚举字符串 ss 中的每个位置作为起始位置，并判断对应的子串是否为 abcd 即可。

如果 ss 中多次出现了某个子串，那么我们选择最早出现的位置（贪心），可以使得后续子串能被找到的机会更大。

开头或末尾不是∗，则直接匹配。

time: O(mn)

space: O(1)